In [68]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import copy

import sys
sys.path.append("Fall 2024")

import kmax as kx
import utility as util
from hierarchy import *

# Continuous Edge Weights - Transition Probabilities

## Function Definitions

## Simulation

In [2]:
num_samples = 1000
lam = 0.7
offspring_dist = lambda: np.random.poisson(lam)
edge_dist = lambda: np.random.uniform(0,1)

In [34]:
# Setup transition count matrix
num_sections = 4
transition_counts = np.zeros(shape=(num_sections+1,num_sections+1),
                             dtype=int)

In [64]:
def get_section(Z_val):
    if Z_val <= 0:
        return 0
    else:
        section_bounds = np.linspace(0.0, 1.0, num_sections)
        sec = 0
        while section_bounds[sec] < Z_val:
            sec += 1
        return sec

In [66]:
for _ in range(num_samples):
    # Generate tree
    tree = nx.Graph(GWBP(dist=offspring_dist, MAXLEVEL=10))
    for u,v in tree.edges():
        tree[u][v]['weight'] = edge_dist()

    if tree.number_of_nodes() == 1:
        transition_counts[0][0] += 1
        continue
    
    # Get Z values, children
    tups = util.path_cover(tree, root=0, return_tuples=True)
    Z = {v:tup[1] for v,tup in tups}
    v1 = {v:tup[2] for v,tup in tups}
    v2 = {v:tup[3] for v,tup in tups}

    current_node = v1[0]
    while True:
        current_section = get_section(Z[current_node])
        if v1[current_node] is not None:
            v1_section = get_section(Z[v1[current_node]])
            transition_counts[current_section][v1_section] += 1
            current_node = v1[current_node]
        else:
            transition_counts[current_section][0] += 1
            break

In [67]:
transition_counts

array([[4538,    0,    7,   27,    0],
       [ 161,   38,   47,   59,    0],
       [ 180,   47,   50,   43,    0],
       [ 201,   66,   56,   35,    0],
       [   0,    0,    0,    0,    0]])

In [82]:
my_shape = transition_counts.shape
transition_probs = np.ndarray(shape=my_shape, dtype=float)
print(transition_probs)
for i in range(my_shape[0]):
    for j in range(my_shape[1]):
        transition_probs[i][j] = transition_counts[i][j] / num_samples
print(transition_probs)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[4.538 0.    0.007 0.027 0.   ]
 [0.161 0.038 0.047 0.059 0.   ]
 [0.18  0.047 0.05  0.043 0.   ]
 [0.201 0.066 0.056 0.035 0.   ]
 [0.    0.    0.    0.    0.   ]]
